# 11.02 - Polars Foundations: The Expression Engine

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Avanzado  
**Tiempo estimado:** 45 min

Polars no es solo un clon de Pandas más rápido; es un cambio de paradigma en cómo se procesan los datos. En este lab exploraremos los cimientos de su motor de expresiones.

In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

print(f"Polars version: {pl.__version__}")

Polars version: 1.29.0


## 1. El Sistema de Tipos y Series
A diferencia de Pandas, Polars tiene un sistema de tipos estricto basado en Apache Arrow.

In [2]:
# Creación de Series con tipos explícitos
s = pl.Series("counts", [1, 2, 3], dtype=pl.Int32)
print(s)

# Los nulos en Polars (null) no son NaN de punto flotante
s_nulls = pl.Series("data", [1, None, 3])
print(f"Has nulls: {s_nulls.has_nulls()}")
print(s_nulls)

shape: (3,)
Series: 'counts' [i32]
[
	1
	2
	3
]
Has nulls: True
shape: (3,)
Series: 'data' [i64]
[
	1
	null
	3
]


## 2. El Corazón de Polars: Expresiones
Las expresiones en Polars son declarativas. No se ejecutan hasta que se añaden a un contexto (`select`, `with_columns`, `group_by`).

In [3]:
df = pl.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "David"],
    "age": [25, 30, 35, 40],
    "salary": [50000, 60000, 70000, 80000]
})

# Una expresión es un objeto plano de ejecución
expr = pl.col("age") * 2
print(type(expr))

# Aplicando expresiones en contextos
df.select([
    pl.col("name").str.to_uppercase(),
    pl.col("salary").mean().alias("avg_salary"),
    (pl.col("age") / 2).alias("half_age")
])

<class 'polars.expr.expr.Expr'>


name,avg_salary,half_age
str,f64,f64
"""ALICE""",65000.0,12.5
"""BOB""",65000.0,15.0
"""CHARLIE""",65000.0,17.5
"""DAVID""",65000.0,20.0


## 3. Contextos: Select vs With Columns
- `select()`: Crea un nuevo DataFrame con solo las columnas resultantes.
- `with_columns()`: Mantiene las columnas existentes y añade o modifica otras.

In [4]:
df_new = df.with_columns([
    (pl.col("salary") * 1.1).alias("salary_bonus"),
    pl.when(pl.col("age") > 30).then(pl.lit("Senior")).otherwise(pl.lit("Junior")).alias("level")
])
df_new

name,age,salary,salary_bonus,level
str,i64,i64,f64,str
"""Alice""",25,50000,55000.0,"""Junior"""
"""Bob""",30,60000,66000.0,"""Junior"""
"""Charlie""",35,70000,77000.0,"""Senior"""
"""David""",40,80000,88000.0,"""Senior"""


## 4. Selección Dinámica de Columnas
Polars permite seleccionar columnas por tipo o patrones, algo muy útil en datasets grandes.

In [5]:
# Seleccionar todas las columnas numéricas y calcular su media
df_new.select([
    pl.col(pl.Int64, pl.Float64).mean()
])

age,salary,salary_bonus
f64,f64,f64
32.5,65000.0,71500.0


## Resumen
- Las **expresiones** son el bloque de construcción.
- Los **contextos** definen dónde se aplican las expresiones.
- El sistema de tipos es más robusto y predecible que en Pandas.

---

**Anterior:** [10.03 - Tendencias y Patrones](../10_temporal_analytics/10_03_trends_and_patterns.ipynb)  
**Siguiente:** [11.03 - Polars Wrangling](./11_03_polars_wrangling.ipynb)